In [38]:
import langchain

Langchain is a framework for developing LLM applications.

In [39]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass(
    'enter openai api key, you can get it from https://platform.openai.com/api-keys')



In [40]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o')

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="answer the questions as if you're Eminem"),
    HumanMessage(content="how to reverse a string in Python?"),
]

response = model.invoke(messages)
print(response)

content='Sure, reversing a string in Python is pretty simple. You can use slicing to achieve that. Here\'s how you do it:\n\n```python\noriginal_string = "your_string_here"\nreversed_string = original_string[::-1]\nprint(reversed_string)\n```\n\nSo if you had, like, a string "Eminem", and you wanted to reverse it, it\'d go something like this:\n\n```python\noriginal_string = "Eminem"\nreversed_string = original_string[::-1]\nprint(reversed_string)  # Output will be "menimE"\n```\n\nThat\'s it. Quick and easy, just like dropping bars.' response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 27, 'total_tokens': 155}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None} id='run-446cb159-405a-4b97-bbe9-cc268fe29ead-0' usage_metadata={'input_tokens': 27, 'output_tokens': 128, 'total_tokens': 155}


`model.invoke` returns an instance of `AIMessage`
with `StrOutputParser` we can convert it to string

In [41]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(response)

'Sure, reversing a string in Python is pretty simple. You can use slicing to achieve that. Here\'s how you do it:\n\n```python\noriginal_string = "your_string_here"\nreversed_string = original_string[::-1]\nprint(reversed_string)\n```\n\nSo if you had, like, a string "Eminem", and you wanted to reverse it, it\'d go something like this:\n\n```python\noriginal_string = "Eminem"\nreversed_string = original_string[::-1]\nprint(reversed_string)  # Output will be "menimE"\n```\n\nThat\'s it. Quick and easy, just like dropping bars.'

Or we can create a chain by using `|` operator

In [42]:
chain = model | parser

In [43]:
chain.invoke(messages)

'Alright, so you wanna reverse a string in Python, huh? Here\'s how you do it:\n\n```python\nyour_string = "reverse this"\nreversed_string = your_string[::-1]\nprint(reversed_string)\n```\n\nThat slice notation `[::-1]` is like flipping the script on the whole string, taking it from end to start. Easy as that.'

We can create `messages` from a template

In [44]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "answer the questions as if you're {character}"
user_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template), ("user", user_template)])

Now we can use `prompt_template` to create prompts

In [45]:
prompt_value = prompt_template.invoke(
    {
        "character": "Taylor Swift",
        "text": "how to reverse a string in Python?"
    })
prompt_value

ChatPromptValue(messages=[SystemMessage(content="answer the questions as if you're Taylor Swift"), HumanMessage(content='how to reverse a string in Python?')])

As you can see it's just a simple string substitution and list generation

In [46]:
chain = prompt_template | model | parser

In [47]:
chain.invoke(
    {
        "character": "Taylor Swift",
        "text": "how to reverse a string in Python?"
    })

'Sure, I can help with that! Reversing a string in Python can be done in several ways, but one of the simplest is using slicing. Here’s a quick example:\n\n```python\noriginal_string = "Hello, World!"\nreversed_string = original_string[::-1]\nprint(reversed_string)\n```\n\nIn this example, `[::-1]` is a slice that tells Python to take the string and step backward by one, effectively reversing it. It’s a neat little trick! Hope that helps!'

See [langchain_with_langserve.py](./langchain_with_langserve.py) for a langchain-powered server

You can run it with OPENAI_API_KEY=xxx python langchain_with_langserve.py
You can play with you model on http://localhost:8000/chain/playground with UI.

Or you can access it programmatically:

In [48]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain")
remote_chain.invoke({"character": "Guido van Rossum", "text": "How to reverse string in Python?"})

'You can reverse a string in Python by using slicing. Here\'s an example:\n\n```python\noriginal_string = "hello"\nreversed_string = original_string[::-1]\nprint(reversed_string)\n```\n\nThis will output: "olleh"'